In [1]:
import SimpleITK as sitk
import pdb
import click
import os
from os.path import join
import numpy as np
import pandas as pd
import nibabel as nib
import cv2
import csv
from scipy.ndimage import gaussian_filter
from scipy.ndimage.measurements import label
from scipy.ndimage.morphology import generate_binary_structure
from scipy.ndimage.measurements import center_of_mass
%matplotlib nbagg
#import gui
#import registration_gui as rgui

input_path = '../../data/hecktor_nii/'
output_path = '../../data/bbox/'
output_shape = (144, 144, 144)

In [2]:
def write_nii(wrt, img, path):
    wrt.SetFileName(path)
    wrt.Execute(img)

def check_singleGTVt(gtvt):
    s = generate_binary_structure(2,2)
    labeled_array, num_features = label(gtvt)
    if num_features !=1:
        print('num_features-------------------------------',num_features)
        print('number of voxels:')
        for i in np.unique(labeled_array)[1:]:
            print (np.sum(labeled_array==i))
        print('centers:')
        for i in np.unique(labeled_array)[1:]:
            print (center_of_mass(labeled_array==i))
    return 0
    

def bbox_auto(vol_pt, gtvt, px_spacing_pt, px_spacing_ct, px_origin_pt, px_origin_ct, output_shape=(144, 144, 144), th = 3, auto_th = False, bbox=None):
    # We find the oropharynx region from the PET based on brain segmentation
    output_shape_pt = tuple(e1 // e2 for e1, e2 in zip(output_shape, px_spacing_pt)) 
    # Gaussian smooth
    vol_pt_gauss = gaussian_filter(vol_pt, sigma=3)
    # auto_th: based on max SUV value in the top of the PET scan, for some cases that have unusual SUV values
    if auto_th: 
        th = np.max(vol_pt[np.int(vol_pt.shape[0] * 2 // 3):, :, :]) / 2.6
        print ('auto_th = ', th, '----------------------------------')
    # OR fixed threshold (for all other cases)
    vol_pt_thgauss = np.where(vol_pt_gauss > th, 1, 0)
    # Find brain as biggest blob AND not in lowest third of the scan
    labeled_array, _ = label(vol_pt_thgauss)
    try:
        vol_pt_brain = labeled_array == np.argmax(np.bincount(labeled_array[vol_pt.shape[0] * 2 // 3:].flat)[1:]) + 1
    except: 
        print('th too high?')
        # Quick fix just to pass for all cases
        th = 0.1
        vol_pt_thgauss = np.where(vol_pt_gauss > th, 1, 0)
        labeled_array, _ = label(vol_pt_thgauss)
        vol_pt_brain = labeled_array == np.argmax(np.bincount(labeled_array[vol_pt.shape[0] * 2 // 3:].flat)[1:]) + 1
    # Find lowest voxel of the brain and box containing the brain
    z = np.min(np.argwhere(np.sum(vol_pt_brain, axis=(1, 2))))
    y1 = np.min(np.argwhere(np.sum(vol_pt_brain, axis=(0, 2))))
    y2 = np.max(np.argwhere(np.sum(vol_pt_brain, axis=(0, 2))))
    x1 = np.min(np.argwhere(np.sum(vol_pt_brain, axis=(0, 1))))
    x2 = np.max(np.argwhere(np.sum(vol_pt_brain, axis=(0, 1))))
    # slice = vol_pt_brain[np.min(np.argwhere(np.sum(vol_pt_brain, axis=(1, 2))))]
    # x, y = center_of_mass(slice)
    
    # Center bb based on this
    zshift = 30//px_spacing_pt[2]
    if z - (output_shape_pt[2] - zshift) < 0:
        zbb = (0, output_shape_pt[2])
    elif z + zshift > vol_pt.shape[0]:
        zbb = (vol_pt.shape[0] - output_shape_pt[2], vol_pt.shape[0])
    else:
        zbb = (z - (output_shape_pt[2] - zshift), z + zshift)

    yshift = 30//px_spacing_pt[1]
    if np.int((y2 + y1) / 2 - yshift - np.int(output_shape_pt[1] / 2)) < 0:
        ybb = (0, output_shape_pt[1])
    elif np.int((y2 + y1) / 2 - yshift - np.int(output_shape_pt[1] / 2)) > vol_pt.shape[1]:
        ybb = vol_pt.shape[1] - output_shape_pt[1], vol_pt.shape[1]
    else:
        ybb = (np.int((y2 + y1) / 2 - yshift - np.int(output_shape_pt[1] / 2)), np.int((y2 + y1) / 2 - yshift + np.int(output_shape_pt[1] / 2)))

    if np.int((x2 + x1) / 2 - np.int(output_shape_pt[0] / 2)) < 0:
        xbb = (0, output_shape_pt[0])
    elif np.int((x2 + x1) / 2 - np.int(output_shape_pt[0] / 2)) > vol_pt.shape[2]:
        xbb = vol_pt.shape[2] - output_shape_pt[0], vol_pt.shape[2]
    else:
        xbb = (np.int((x2 + x1) / 2 - np.int(output_shape_pt[0] / 2)), np.int((x2 + x1) / 2 + np.int(output_shape_pt[0] / 2)))

    print(zbb, ybb, xbb)
    z_pt = np.asarray(zbb,dtype=np.int)
    y_pt = np.asarray(ybb,dtype=np.int)
    x_pt = np.asarray(xbb,dtype=np.int)

    # In the physical dimensions
    z_abs = z_pt * px_spacing_pt[2] + px_origin_pt[2]
    y_abs = y_pt * px_spacing_pt[1] + px_origin_pt[1]
    x_abs = x_pt * px_spacing_pt[0] + px_origin_pt[0]
    
    # In the CT resolution:
    z_ct = np.asarray((z_abs-px_origin_ct[2])//px_spacing_ct[2],dtype=np.int)
    y_ct = np.asarray((y_abs-px_origin_ct[1])//px_spacing_ct[1],dtype=np.int)
    x_ct = np.asarray((x_abs-px_origin_ct[0])//px_spacing_ct[0],dtype=np.int)
    print(z_ct,y_ct,x_ct)

    # Check that the bbox contains the tumors
    fail = False
    if np.sum(gtvt[z_ct[0]:z_ct[1], y_ct[0]:y_ct[1], x_ct[0]:x_ct[1]]) != np.sum(gtvt):
        print('GTVt outside bbox ------------------------------------')
        fail = True
    # Add the fails for which we had to change the threshold to keep track
    if auto_th:
        fail = True
    
    
    if bbox is not None:
        x_abs = bbox[0:2]
        y_abs = bbox[2:4]
        z_abs = bbox[4:6]
        
        z_pt = np.asarray((z_abs - px_origin_pt[2])/ px_spacing_pt[2],dtype=np.int)
        y_pt = np.asarray((y_abs - px_origin_pt[1])/ px_spacing_pt[1],dtype=np.int)
        x_pt = np.asarray((x_abs - px_origin_pt[0])/ px_spacing_pt[0],dtype=np.int)
        
        z_ct = np.asarray((z_abs-px_origin_ct[2])//px_spacing_ct[2],dtype=np.int)
        y_ct = np.asarray((y_abs-px_origin_ct[1])//px_spacing_ct[1],dtype=np.int)
        x_ct = np.asarray((x_abs-px_origin_ct[0])//px_spacing_ct[0],dtype=np.int)
        #x_pt = np.asarray([50,76],dtype=np.int)
        #y_pt = np.asarray([43,70],dtype=np.int)
        #z_pt = np.asarray([212,256],dtype=np.int)
        #print (x_pt,y_pt,z_pt)
        #z_abs = z_pt * px_spacing_pt[2] + px_origin_pt[2]
        #y_abs = y_pt * px_spacing_pt[1] + px_origin_pt[1]
        #x_abs = x_pt * px_spacing_pt[0] + px_origin_pt[0]
        #pdb.set_trace()
        
        if np.sum(gtvt[z_ct[0]:z_ct[1], y_ct[0]:y_ct[1], x_ct[0]:x_ct[1]]) != np.sum(gtvt):
            print('still GTVt outside bbox ------------------------------------')
        else: 
            print('now GTVt inside bbox ------------------------------------')
    
    # Plot box on vol_pt_brain for visualization
    vol_pt_brain[z_pt[0]:z_pt[1], y_pt[0]:y_pt[0] + 1, x_pt[0]:x_pt[1]] = True
    vol_pt_brain[z_pt[0]:z_pt[1], y_pt[1]:y_pt[1] + 1, x_pt[0]:x_pt[1]] = True

    vol_pt_brain[z_pt[0]:z_pt[1], y_pt[0]:y_pt[1], x_pt[0]:x_pt[0] + 1] = True
    vol_pt_brain[z_pt[0]:z_pt[1], y_pt[0]:y_pt[1], x_pt[1]:x_pt[1] + 1] = True

    vol_pt_brain[z_pt[0]:z_pt[0] + 1, y_pt[0]:y_pt[1], x_pt[0]:x_pt[1]] = True
    vol_pt_brain[z_pt[1]:z_pt[1] + 1, y_pt[0]:y_pt[1], x_pt[0]:x_pt[1]] = True
    
        
    return vol_pt_brain, fail, z_abs, y_abs, x_abs


def clip(vol, clip_values=None):
    # We clip the CT values
    if clip_values:
        vol[vol < clip_values[0]] = clip_values[0]
        vol[vol > clip_values[1]] = clip_values[1]
    return vol


In [3]:
try:
    os.mkdir(output_path)
    print("Directory ", output_path, " Created ")
except FileExistsError:
    print("Directory ", output_path, " already exists")

writer = sitk.ImageFileWriter()
writer.SetImageIO("NiftiImageIO")

with open(join(output_path,'../bbipynb.csv'), 'w', newline='') as csvfile:
    bbwrite = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    bbwrite.writerow(['PatientID', 'x1', 'x2', 'y1', 'y2', 'z1', 'z2'])

patients = []
for f in sorted(os.listdir(input_path)):
    patients.append(f.split('_')[0])
nfail = 0
n=0
list_auto_th = ['CHUM010','CHUS021','CHGJ026','CHMR023','CHGJ053','CHMR028']
list_fix_bb = ['CHMR028','CHGJ053','CHGJ082']
dict_fix_bb = {
    "CHMR028": np.asarray([-73.828125,68.359375,-112.109375,35.546875,-204.0536231994629,-60.17230224609375
]),
    "CHGJ053": np.asarray([-86.1328125,54.4921875,-166.9921875,-26.3671875,-214.2802734375,-70.4007568359375]),
    "CHGJ082": np.asarray([-68.5546875,72.0703125,-170.5078125,-29.8828125,-245.0201416015625,-101.140625])
          }
for patient in patients:
    #list_p = ['HN-CHUM-020','HN-CHUM-026','HN-CHUM-030','HN-CHUM-042','HN-CHUM-053','HN-CHUM-057','HN-CHUM-065','HN-CHUS-010','HN-CHUS-035','HN-CHUS-045','HN-CHUS-057','HN-CHUS-074','HN-CHUS-086','HN-CHUS-096','HN-HGJ-025','HN-HGJ-062','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053','HN-CHUM-053']
    # # pdb.set_trace()
    #if patient not in list_auto_th[:4]:
    #    continue
    #if patient not in ['HN-HMR-028','HN-HGJ-053','HN-HGJ-082']:
    #    continue
    print('************* patient:', patient)
    in_path_ct = input_path + patient + '/' + patient + '_ct.nii.gz'
    in_path_gtvt_roi = input_path + patient + '/' + patient + '_ct_gtvt.nii.gz'
    if not os.path.exists(in_path_gtvt_roi):
        print('no GTVt')
    in_path_gtvn_roi = input_path + patient + '/' + patient + '_ct_gtvn.nii.gz'
    in_path_pt = input_path + patient + '/' + patient + '_pt.nii.gz'
    #out_path_bbox = output_path + patient + '/' + patient + '_ct_bbox'
    try: 
        img_ct = sitk.ReadImage(in_path_ct)
        img_pt = sitk.ReadImage(in_path_pt)
    except:
        print('cannot read ------------')
        continue
    px_spacing_ct = img_ct.GetSpacing()
    px_spacing_pt = img_pt.GetSpacing()
    px_origin_ct = img_ct.GetOrigin()
    px_origin_pt = img_pt.GetOrigin()
    img_ct = sitk.GetArrayFromImage(img_ct)
    gtvt = sitk.GetArrayFromImage(sitk.ReadImage(in_path_gtvt_roi))
    check_singleGTVt(gtvt)
    #gtvn = sitk.GetArrayFromImage(sitk.ReadImage(in_path_gtvn_roi))
    img_pt = sitk.GetArrayFromImage(sitk.ReadImage(in_path_pt))
    
    # Fix threshold for some of the patients:
    auto_th = False
    if patient in list_auto_th[:4]:
        auto_th = True
    # Fix directly the bbox for some that don't work
    bbox = None
    if patient in list_fix_bb:
        bbox = dict_fix_bb[patient]
    img_brain, fail, z_bb, y_bb, x_bb = bbox_auto(img_pt, gtvt, px_spacing_pt, px_spacing_ct, px_origin_pt, px_origin_ct, output_shape, auto_th=auto_th, bbox=bbox)
    nfail = nfail + fail
    n = n + 1
    perm = (0, 1, 2)  # No permutation needed now
    img_brain = sitk.GetImageFromArray(np.transpose(img_brain.astype(np.uint8), perm), isVector=False)
    # img_pt = sitk.GetImageFromArray(np.transpose(img_pt, perm), isVector=False)
    out_path_brain = output_path + patient + '_brain.nii'
    write_nii(writer, img_brain, out_path_brain)
    
    # Write bb position in csv. To change to panda frame
    with open(join(output_path,'../bbipynb.csv'), 'a', newline='') as csvfile:
        bbwrite = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        bbwrite.writerow([patient, str(x_bb[0]), str(x_bb[1]), str(y_bb[0]), str(y_bb[1]), str(z_bb[0]), str(z_bb[1])])

print ('fails/total',nfail,n)

Directory  ../../data/bbox/  already exists
************* patient: CHGJ007
(25.0, 69.0) (16, 56) (45, 85)
[25 69] [ 85 229] [189 333]
************* patient: CHGJ008
(24.0, 68.0) (16, 56) (45, 85)
[23 67] [ 85 229] [189 333]
************* patient: CHGJ010
(21.0, 65.0) (25, 65) (43, 83)
[21 65] [117 261] [182 326]
************* patient: CHGJ013
(21.0, 65.0) (20, 60) (42, 82)
[21 65] [ 99 243] [178 322]
************* patient: CHGJ015
(26.0, 70.0) (20, 60) (45, 85)
[26 70] [ 99 243] [189 333]
************* patient: CHGJ016
(10.0, 54.0) (24, 64) (42, 82)
[10 54] [113 257] [178 322]
************* patient: CHGJ017
(27.0, 71.0) (19, 59) (43, 83)
[27 71] [ 95 239] [182 326]
************* patient: CHGJ018
(23.0, 67.0) (12, 52) (43, 83)
[23 67] [ 70 214] [182 326]
************* patient: CHGJ025
(27.0, 71.0) (19, 59) (45, 85)
[27 71] [ 95 239] [189 333]
************* patient: CHGJ026
auto_th =  1.9397539358872633 ----------------------------------
(21.0, 65.0) (23, 63) (43, 83)
[21 65] [110 254] [

(33.0, 68.0) (46, 82) (53, 89)
[ 93 186] [150 298] [179 327]
************* patient: CHUM017
(40.0, 84.0) (37, 77) (42, 82)
[39 83] [160 304] [178 322]
************* patient: CHUM018
(38.0, 82.0) (35, 75) (42, 82)
[37 81] [153 297] [178 322]
************* patient: CHUM019
(34.0, 70.0) (36, 72) (52, 88)
[ 95 191] [109 257] [175 322]
************* patient: CHUM021
(26.0, 70.0) (37, 77) (43, 83)
[25 69] [160 304] [182 326]
************* patient: CHUM022
(32.0, 68.0) (38, 74) (52, 88)
[ 88 184] [118 265] [175 322]
************* patient: CHUM023
(36.0, 72.0) (36, 72) (52, 88)
[102 198] [109 257] [175 322]
************* patient: CHUM024
(31.0, 67.0) (40, 76) (53, 89)
[ 85 181] [126 273] [179 327]
************* patient: CHUM026
(38.0, 82.0) (33, 73) (44, 84)
[37 81] [146 290] [185 329]
************* patient: CHUM027
(18.0, 54.0) (39, 75) (52, 88)
[ 69 165] [122 269] [175 322]
************* patient: CHUM029
(31.0, 67.0) (39, 75) (55, 91)
[ 93 189] [122 269] [187 335]
************* patient: CHUM

(184.0, 220.0) (35, 71) (55, 91)
[38 86] [130 253] [199 322]
************* patient: CHUS065
(179.0, 215.0) (35, 71) (49, 85)
[ 61 133] [130 253] [178 301]
************* patient: CHUS066
(184.0, 220.0) (36, 72) (54, 90)
[44 92] [134 257] [195 318]
************* patient: CHUS067
(166.0, 202.0) (35, 71) (54, 90)
[42 90] [130 253] [195 318]
************* patient: CHUS068
(185.0, 221.0) (37, 73) (53, 89)
[ 62 134] [137 260] [192 315]
************* patient: CHUS069
(185.0, 221.0) (38, 74) (55, 91)
[37 85] [141 264] [199 322]
************* patient: CHUS073
(163.0, 199.0) (35, 71) (53, 89)
[39 87] [130 253] [192 315]
************* patient: CHUS074
(186.0, 222.0) (36, 72) (54, 90)
[44 92] [134 257] [195 318]
************* patient: CHUS076
(183.0, 219.0) (34, 70) (60, 96)
[47 95] [127 250] [216 339]
************* patient: CHUS077
(163.0, 199.0) (31, 67) (53, 89)
[31 79] [117 240] [192 315]
************* patient: CHUS078
(178.0, 214.0) (37, 73) (50, 86)
[42 90] [137 260] [182 304]
************* p